# Feature Engineering

Todos los algoritmos de aprendizaje automático utilizan datos de entrada para crear datos de salida. Estos datos de entrada tienen una serie de características, que normalmente tienen forma de columnas estructuradas (texto, números). Sin embargo cada algoritmo requiere que los datos de entrada cumplan determinadas características, es aquí donde surge la necesidad de la ingeniería de características (*feature engineering*). El *feature engineering* tiene principalmente dos objetivos:

- Adecuar el dataset a los requisitos del modelo o algoritmo
- Mejorar el rendimiento del algoritmo, ya sea a nivel computacional, o en cuanto a la bondad del ajuste

Algunas de estas acciones se deben a la propia naturaleza de la recogida de los datos. En la práctica mayoría de actividades en las que interviene la ciencia de datos, los datos recogidos en los dataset contendrán variables categóricas. Este tipo de variables es usualmente almacenada como texto. Dado que los modelos de machine learning están basados en ecuaciones matemáticas, introducir variables de tipo texto puede ocasionar en la mayoría de casos un resultado erróneo, o que ni si quiera se pueda ejecutar el algoritmo. Otro caso muy común es el tratamiento de outliers o datos extremos, o cómo actuar cuando ciertos datos no están disponibles.

Algunas de las técnicas de *feature engineering* más comunes son:

- Imputación
- Tratamiento de outliers
- Binning
- Encoders
- Scaling
- Log Transform

# 1. Binning

El objetivo principal del *binning* (hacer *bins*, contenedores) es dotar al modelo de mayor robustez y evitar problemas de *overfitting*, aunque a costa del desempeño. Cada vez que se agrupa una variable, se sacrifica información y los datos quedan más regularizados.

>*La regularización en ML evita que el modelo sea muy flexible a los datos de entrada, trando así de evitar el overfitting.*

La clave de esta técnica radica en el balance entre desempeño y *overfitting*. De forma general para variables numéricas, este proceso es un poco redundante. Sin embargo suele ser muy útil cuando se trabaja con variables categóricas, por ejemplo asignando en una categoría general categorías con frecuencias muy bajas.

<center><img src="../_images\ml_fe_binning.png" alt="Drawing" style="width: 200px;"/></center>

# 2. Encoding

En la práctica mayoría de actividades en las que interviene la ciencia de datos, los datos recogidos en los dataset contendrán variables categóricas. Este tipo de variables es usualmente almacenada como texto. Dado que los modelos de machine learning están basados en ecuaciones matemáticas, introducir variables de tipo texto puede ocasionar en la mayoría de casos un resultado erróneo, o que ni si quiera se pueda ejecutar el algoritmo.

El proceso por el que se pasa de una variable categórica a una numérica se denomina *encoding*. Algunos de los encoders mas usuales son:

- Label encoding
- One-hot encoding
- Target encoding
- Frecuency encoding

## 2.1. Label encoding

Es el proceso por el que se asigna a cada valor de una variable categórica, un número aleatorio. Debido a la alatoriedad del proceso, esta técnica no es muy recomendable.

El único caso en el que podría ser útil, es a la hora de codificar variables categóricas ordinales, siempre y cuando se haga de forma sucesiva y no aleatoria. Por ejemplo: frío-1, templado-2, calor-3. Para realizar esta tarea se realizaría mediante un diccionario.

In [5]:
# Creamos un dataset
import pandas as pd

data = {'sex': ['male', 'male', 'female', 'female', 'other']}
df = pd.DataFrame(data)
df

,sex
0,male
1,male
2,female
3,female
4,other


In [6]:
# Cargamos el módulo e instanciamos la clase LabelEncoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Creamos la variable sex_econded
df["sex_label_encoded"] = encoder.fit_transform(df["sex"])
df.head()

,sex,sex_label_encoded
0,male,1
1,male,1
2,female,0
3,female,0
4,other,2


## 2.2. One-Hot encoding

Es uno de los métodos más comunes en ML. Esta técnica expande los diferentes $n$ valores de una variable en $n-1$ columnas y las codifica con uno o cero. Los valores binarios expresan la relación entre la columna original y las codificadas. Esta técnica es similar a otro encoder denominado *get_dummies*, el cual realiza el mismo proceso pero con $n$ columnas.

- Regresiones: usar $n-1$ variables, asegura el número correcto de grados de libertad
- Clasificaciones: usar $n$ variables

Un problema de este método, es que cuando se aplica sobre variables que tienen una cantidad de valores únicos elevada, puede disparar la volumetría del dataset.

In [1]:
# Creamos un dataset
import pandas as pd

data = {'sex': ['male', 'male', 'female', 'female', 'other']}
df = pd.DataFrame(data)
df

,sex
0,male
1,male
2,female
3,female
4,other


In [2]:
sex_cat = pd.get_dummies(
    df,
    columns=['sex'],
    prefix='sex',
    prefix_sep='_'
)

sex_cat

,sex_female,sex_male,sex_other
0,0,1,0
1,0,1,0
2,1,0,0
3,1,0,0
4,0,0,1


## 2.3. Binary encoding

Existe una variante del método anterior, denominado *binary encoding*. Esta técnica, en vez de crear una variable nueva por cada valor único, etiqueta con un número cada valor, lo transforma a formato binario, y el número binario es dividio en diferentes columnas, evitando así problemas de volumetría cuando la variable tiene una cardinalidad elevada.

<center><img src="../_images\ml_fe_binary_encoding.png" alt="Drawing" style="width: 600px;"/></center>

Por ejemplo para una variable con una cardinalidad 100, *one-hot encoding* crearía 100 variables, mientras que *binary encoding* solo 7.

## 2.4. Target encoding - Mean encoding

La idea del target encoding es dada una variable categórica $x$, y un target $y$ que puede ser binario o continuo, calcular la media de $y$, para cada valor de $x$. El target es el valor de salida del dataset que queremos predecir. Este método permite codificar una variable categórica con multitud de variables, sin aumentar la volumetría del dataset.

La mayor desventaja de este método es su dependencia con la distribucion del target, y su menor poder predictivo comparado con el método de codificacion binaria. También puede crear problemas relacionados con el *overfitting*, aunque existen ténicas como la regularización y la validación cruzada que minimizan este efecto.

También es importante tener en cuenta, que esta codificación debe llevarse a cabo después de la separación del dataset en los conjuntos de entrenamiento y test.

<center><img src="../_images\ml_fe_target_encoding.png" alt="Drawing" style="width: 600px;"/></center>

In [36]:
# Creamos un dataset
import pandas as pd

df = pd.DataFrame({
    'x_0': ['a'] * 5 + ['b'] * 5,
    'x_1': ['c'] * 9 + ['d'] * 1,
    'y': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
})
df

,x_0,x_1,y
0,a,c,1
1,a,c,1
2,a,c,1
3,a,c,1
4,a,c,0
5,b,c,1
6,b,c,0
7,b,c,0
8,b,c,0
9,b,d,0


In [37]:
means = df.groupby('x_0')['y'].mean()
means

x_0
a    0.8
b    0.2
Name: y, dtype: float64

In [38]:
df['x_0_te'] = df['x_0'].map(means)
df

,x_0,x_1,y,x_0_te
0,a,c,1,0.8
1,a,c,1,0.8
2,a,c,1,0.8
3,a,c,1,0.8
4,a,c,0,0.8
5,b,c,1,0.2
6,b,c,0,0.2
7,b,c,0,0.2
8,b,c,0,0.2
9,b,d,0,0.2


In [39]:
df['x_1_te'] = df['x_1'].map(df.groupby('x_1')['y'].mean())
df

,x_0,x_1,y,x_0_te,x_1_te
0,a,c,1,0.8,0.555556
1,a,c,1,0.8,0.555556
2,a,c,1,0.8,0.555556
3,a,c,1,0.8,0.555556
4,a,c,0,0.8,0.555556
5,b,c,1,0.2,0.555556
6,b,c,0,0.2,0.555556
7,b,c,0,0.2,0.555556
8,b,c,0,0.2,0.555556
9,b,d,0,0.2,0.000000


## 2.5. Frequency encoding

Esta técnica utiliza la frecuencia con la que aparece cada valor único dentro de una variable categórica, para codificar ésta.

In [10]:
# Creamos un dataset
import pandas as pd

data = {'sex': ['male', 'male', 'female', 'female', 'female', 'other']}
df = pd.DataFrame(data)
df

,sex
0,male
1,male
2,female
3,female
4,female
5,other


In [11]:
fe = df.groupby('sex').size()/len(df)
df['sex_fe'] = df['sex'].map(fe)
df

,sex,sex_fe
0,male,0.333333
1,male,0.333333
2,female,0.500000
3,female,0.500000
4,female,0.500000
5,other,0.166667


# 3. Log Transform

La transformación logarítimicas es una de las transformaciones matemáticas más utilizadas en la ingeniería de características. Es utilizada en variables continuas que no tengan una distribución uniforme. Las ventajas principales son:

- Normaliza distribuciones de variables que tengan una asimetría marcada
- Minimiza el efecto de outliers debido a la normalización de valores extremos

Hay que asegurar que los datos sean todos positivos, si no el logaritmo puede dar error.

In [16]:
#Log Transform Example
import numpy as np

data = pd.DataFrame({'value':[2,45, -23, 85, 28, 2, 35, -12]})
data['log(x+1)'] = (data['value']+1).transform(np.log)

#Negative Values Handling
#Note that the values are different
data['log(x-min(x)+1)'] = (data['value']-data['value'].min()+1).transform(np.log)

data

,value,log(x+1),log(x-min(x)+1)
0,2,1.098612,3.258097
1,45,3.828641,4.234107
2,-23,NaN,0.000000
3,85,4.454347,4.691348
4,28,3.367296,3.951244
5,2,1.098612,3.258097
6,35,3.583519,4.077537
7,-12,NaN,2.484907


# 4. Scaling

Aunque en la vida real es normal que el rango de la variables difiera de forma considerable en función de la misma, en ML puede afectar de forma significativa a modelos basados en distancias como k-NN o k-means. Por ejemplo en un modelo que tenga en cuenta el peso (kg) y la altura (cm), la variable altura influenciará mucho más al modelo por tener unos valores más altos.

Para solucionar este tipo de inconvenientes, se pueden utilizar herramientas de escalado, que lo que hacen es igualar el rango en el que se mueven diferentes variables, para que ninguna predomine sobre el resto.

Existen dos tipos de normalizaciones:

- Escalado: modificar el rango de los valores.
- Estandarización: aproximar la distribución de la variable a la de una distribución normal

## 4.1. Escalado min-max

También conocido como normalización min-max, escala todos los valores en un rango fijo entre 0 y 1. Esta transformación no modifica la distribución de la variable, y debido a la minimización de la desviación estándar, el efecto de los outliers se magnifica. Por esto, se recomienda realizar el tratamiento de outliers antes de aplicar esta técnica.

$$X_{norm} = \frac{X - X_{min}}{X_{max}-X_{min}}$$

<center><img src="../_images\ml_fe_minmax.png" alt="Drawing" style="width: 350px;"/></center>

## 4.2. Escalado medio

Es similar al anterior, pero utilizando el valor medio de la variable en vez de el mínimo. Este tipo de escalado modifica la forma de la distribución.

$$X_{norm} = \frac{X - average(x)}{X_{max}-X_{min}}$$

<center><img src="../_images\ml_fe_standarization.png" alt="Drawing" style="width: 350px;"/></center>

## 4.3. Escalado robusto

A cada valor de la variable, se le resta la mediana, y se divide entre el rango intercuartílico. Este tipo de escalado no determina un rango fijo en las distribuciones de salida.

La ventaja de este tipo de escalado es que mitiga la influencia de valores extremos.

$$X_{norm} = \frac{X-median(x)}{IQ_r}$$

## 4.4. Estandarización

La estandarización, o normalización z-score, escala el valor de una variable teniendo en cuenta la desviación estándar. Esto reduce el efecto de los outliers en las características.

La distribución resultante tiene media 0, y distribución estándar 1.

$$z = \frac{x - \mu}{\sigma}$$

Wrap
Use MinMaxScaler as the default if you are transforming a feature. It’s non-distorting.
You could use RobustScaler if you have outliers and want to reduce their influence. However, you might be better off removing the outliers, instead.
Use StandardScaler if you need a relatively normal distribution.

Texto:
- The lame approach: pasar de variables categóricas ordinales (bajo, medio, alto) a numéricas: 1, 2, 3
- Bag of words: se estudia en detalle en NLP

# X. Bibliografía

- https://www.datacamp.com/community/tutorials/encoding-methodologies
- https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114